In [ ]:
import sys
sys.float_info
#Should reveal that floats have 8 bytes

In [ ]:
MD = "metaData"
Indep = "trainIndependents"
Dep = "trainDependent"
fileNames = [MD, Indep, Dep]

In [ ]:
import math ##For basic mathematical operations
import matplotlib.pyplot as plt
import dask.dataframe as dd  
from IPython import display ## Plot setup for Ipython
from matplotlib import cm ##  Colormap reference
from matplotlib import gridspec ##plot setups
from matplotlib import pyplot as plt ##plot setups
import numpy as np 
import pandas as pd
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import collections
import warnings
# warnings.filterwarnings("ignore") - may want to ignore, if know what you are doing, but can be helpful
# warnings.filterwarnings(action = "once") - may want to show only once
# from Queue import priority_queue

In [ ]:
#LOAD FRAMES WITH CSV CONTENTS FOR 0: INDEPENDENTS 1: DEPENDENTS
frames = []
for index, fileName in tqdm(enumerate(fileNames)):
    frame = dd.read_csv("./TestData/" + fileName + ".csv", sep=",", sample = 2000000)
    if(fileName != 'metaData'):
        frames.append(frame)

In [ ]:
#HANDLES NAs ! 
#about 8 seconds
SPECIAL_VAL = 69
for i in tqdm(range(len(frames))):
    frames[i] = frames[i].fillna(SPECIAL_VAL)

In [ ]:
X = frames[0].iloc[:, 1:].to_dask_array(lengths=True) 
y = frames[1].iloc[:, 1:].to_dask_array(lengths=True)

X = np.array(X)
y = np.array(y)

#data split, difference between row indices yields the number of TIMES included in the dataset
xtrain = X[14:16, :]
ytrain = y[14:94, :]

xtest = X[94:114, :]
ytest = y[94:114, :]

In [ ]:
NUM_INDEPENDENT = 20
NUM_SYMBOLS = 1154
#how many times "20070103_13500" to include in building the symbol training sets, must be precise to avoid training on extra zeros
num_filled_times = 2

symbol_training_sets = np.zeros(shape=(NUM_SYMBOLS, num_filled_times), dtype = tuple)

for i in range(num_filled_times):
    if(xtrain[i][0] == SPECIAL_VAL):
        np.delete(symbol_training_sets, i, axis=0)
        i = i - 1
        num_filled_times = num_filled_times - 1
    else:
        for j in range(NUM_SYMBOLS):
            independents = []
            #print(independents)
            for k in range(NUM_INDEPENDENT):
                independents.append(xtrain[i][j*NUM_INDEPENDENT + k])     
            symbol_training_sets[j,i] =(independents, ytrain[i][j])

symbol = 0
#prints a training set for the given symbol
print(symbol_training_sets[symbol]) 
